In [2]:
import os
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout2d
from torch.optim import Adam, SGD
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from skimage.transform import rotate, AffineTransform, warp
import skimage.io as io
import ctypes
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.transforms as transforms
from scipy.ndimage import rotate
from random import random
np.set_printoptions(threshold=np.inf)

Loaing the dataset which was made in the previous notebook

In [3]:
images = np.load("X_50.npy")
labels = np.load("Y_50.npy")
images = images.reshape((images.shape[0],1,50,50))

In [4]:
v=[0,0,0,0]
for i in labels:
    v[i]+=1
print("number of dataapoint in each class is ", v)

number of dataapoint in each class is  [188469, 35755, 70384, 48755]


In [5]:
print("total number of data poinsts", images.shape[0])

total number of data poinsts 343363


The dataset just loaded is highly unbalanced. So during augmentation we augment less occuring class more frequently and more occuring class less frequently so that it can become balanced

In [18]:
# vector X and Y contains all the training images and labels of the augmentatied dataset
train_img = []
Y = []
for itr in tqdm(range(len(labels))):
    p ={'0':[.25,.25,.25,.25],
        '1':[1,1,1,1],
        '2':[.7,.7,.7,.7],
        '3':[1,1,1,1]}
    l = str(labels[itr])
    pr = random()
    if pr < p[l][0]:
        train_img.append(images[itr,:,:])
        Y.append(labels[itr])

    pr = random()
    if pr<p[l][1]:
        train_img.append(cv2.rotate(images[itr,:,:], cv2.ROTATE_180))
        Y.append(labels[itr])
    pr = random()
    if pr<p[l][2]:
        temp=[]
        temp.append(rotate(images[itr,:,:][0],90))
        train_img.append(temp)
        Y.append(labels[itr])
        
    pr = random()
    if pr<p[l][3]:
        temp=[]
        temp.append(rotate(images[itr,:,:][0],270))
        train_img.append(temp)
        Y.append(labels[itr])    
train_img = np.array(train_img)
Y = np.array(Y)

100%|██████████| 10/10 [00:00<00:00, 1701.96it/s]


In [19]:
train_x, val_x, train_y, val_y = train_test_split(train_img,Y, test_size = 0.3, shuffle = True)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)
v=[0,0,0,0]
for i in Y:
    v[i]+=1
print("number of dataapoint in each class are ", v)

number of dataapoint in each class are  [10, 0, 11, 0]


Now the dataset is approximately balanced

In [22]:
train_y = train_y.astype(int)
val_y = val_y.astype(int)

In [23]:
# These classes helps to load data and implement batch wise training
class TrainHelper(Dataset):
    def __init__(self):
        self.len = train_x.shape[0]
        self.train_x, self.train_y = train_x.astype('float32'), train_y.astype('float32')
        self.train_x = torch.from_numpy(train_x)
        self.train_y = torch.from_numpy(train_y)
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        return self.train_x[idx], self.train_y[idx]
class ValHelper(Dataset):
    def __init__(self):
        self.len = val_x.shape[0]
        self.val_x, self.val_y = val_x.astype('float32'), val_y.astype('float32')
        self.val_x = torch.from_numpy(val_x)
        self.val_y = torch.from_numpy(val_y)
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        return self.val_x[idx], self.val_y[idx]
traindataset = TrainHelper()
valdataset = ValHelper()
train_loader = DataLoader(dataset = traindataset, batch_size=32,shuffle=True,num_workers=0)
val_loader = DataLoader(dataset = valdataset, batch_size=32,shuffle=True,num_workers=0)

In [79]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 64, kernel_size=10, stride=1, padding=0),
            ReLU(inplace=True),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(64, 128, kernel_size=3, stride=1, padding=2),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout2d(0.10),
            
            Conv2d(128, 128, kernel_size=2, stride=1, padding=2),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),         
            Dropout2d(0.20),
            
            Conv2d(128,6272, kernel_size=7, padding=0),   # This technique is called fully connected layer using convulation layer.
            ReLU(inplace=True),
            
            Conv2d(6272, 4, kernel_size=1),
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        return x
class Net2(Module):   
    def __init__(self):
        super(Net2, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 64, kernel_size=10, stride=1, padding=0),
            ReLU(inplace=True),
            BatchNorm2d(64),
            MaxPool2d(kernel_size=2, stride=2),

            Conv2d(64, 128, kernel_size=3, stride=1, padding=2),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Dropout2d(0.10),
            
            Conv2d(128, 256, kernel_size=2, stride=1, padding=2),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),         
            Dropout2d(0.20),
            
            Conv2d(256,12544, kernel_size=7, padding=0),   # This tenique is called fully connected layer using convulation layer.
            ReLU(inplace=True),
            
            Conv2d(12544, 4, kernel_size=1),
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        return x

In [80]:
arr=np.random.randn(1,1,50,50)
arr=torch.from_numpy(arr).float()
model = Net2()
summary(model,(1,50,50))
model(arr).shape

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 4, 1, 1]             --
|    └─Conv2d: 2-1                       [-1, 64, 41, 41]          6,464
|    └─ReLU: 2-2                         [-1, 64, 41, 41]          --
|    └─BatchNorm2d: 2-3                  [-1, 64, 41, 41]          128
|    └─MaxPool2d: 2-4                    [-1, 64, 20, 20]          --
|    └─Conv2d: 2-5                       [-1, 128, 22, 22]         73,856
|    └─ReLU: 2-6                         [-1, 128, 22, 22]         --
|    └─MaxPool2d: 2-7                    [-1, 128, 11, 11]         --
|    └─Dropout2d: 2-8                    [-1, 128, 11, 11]         --
|    └─Conv2d: 2-9                       [-1, 256, 14, 14]         131,328
|    └─ReLU: 2-10                        [-1, 256, 14, 14]         --
|    └─MaxPool2d: 2-11                   [-1, 256, 7, 7]           --
|    └─Dropout2d: 2-12                   [-1, 256, 7, 7]           --
| 

torch.Size([1, 4, 1, 1])

In [ ]:
model = Net2()
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
#Stochastic Gradient Descent is used
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, verbose=True)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
print(model)

In [ ]:
ctr1=0
highest_validation_accuracy = 0
def train(epoch,tb):
    global highest_validation_accuracy
    global ctr1
    # ctr1 are used to count the epoch for the test and training mini-batches
    model.train()
    train_loss = []
    correct = 0
    total = 0
    for batch in train_loader:
        data, target = batch
        x_train, y_train = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            x_train = x_train.cuda()
            y_train = y_train.cuda()
        
        output_train = model(x_train.float())
        output_train = output_train.reshape(-1,4)
        loss_train = criterion(output_train, y_train)
        train_loss.append(loss_train.item()) 
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
        ctr1+=1
        
        _, predicted = torch.max(output_train.data, 1)
        total += y_train.size(0)
        correct += (predicted == y_train).sum().item()
        if ctr1%50==0:
            print("Training accuracy", correct/total)     
    tb.add_scalar("Training Loss", loss_train.item(), ctr1)
    tb.add_scalar("Training accuracy", correct/total,ctr1)     
    

    print("Training Loss on loop",ctr1," is ",  np.mean(train_loss))
    print("Training accuracy on loop",ctr1," is ", correct/total)
    validation_loss = []
    correct = 0
    total = 0
    for batch in val_loader:
        data, target = batch
        x_val, y_val = Variable(data), Variable(target)
        if torch.cuda.is_available():
            x_val = x_val.cuda()
            y_val = y_val.cuda()
        output_val = model(x_val.float())
        output_val =  output_val.reshape(-1,4)
        loss_val = criterion(output_val, y_val)
        validation_loss.append(loss_val.item())            
        _, predicted = torch.max(output_val.data, 1)
        total += y_val.size(0)
        correct += (predicted == y_val).sum().item()
    tb.add_scalar("Validation Loss", np.mean(validation_loss), ctr1)
    tb.add_scalar("Validation accuracy", correct/total,ctr1)
    print("Validation Loss on loop",ctr1," is ",  np.mean(validation_loss))
    print("Validation accuracy on loop",ctr1," is ", correct/total)
    if highest_validation_accuracy-0.005 < correct/total:
            highest_validation_accuracy = correct/total
            name = "ValidationAccuracy"+str(highest_validation_accuracy)+"epoch"+str(ctr1)+'.pth'
            torch.save(model.state_dict(),name)
    
    scheduler.step(correct/total)

In [ ]:
n_epochs = 1000
tb = SummaryWriter(comment = 'second model')
for epoch in range(n_epochs):
    train(epoch,tb)
tb.close()